In [ ]:
from bayes_opt import BayesianOptimization
from run_CartPole import get_average_reward
from sklearn.gaussian_process.kernels import RBF

from bayes_opt.observer import JSONLogger
from bayes_opt.event import Events
from bayes_opt import UtilityFunction

In [ ]:
# black-box function
def black_box_function(lr, e_greedy):
    return get_average_reward(lr,e_greedy)

In [ ]:
dqn_bo = BayesianOptimization(
    f = black_box_function,
    # the hyperparameters to optimizate
    pbounds={'lr': (0.0, 0.2),'e_greedy': (0.5, 1.0),},
    verbose=2,
    random_state=1,
)

In [ ]:
# parameters of gaussian_process
gp_params = {
        'kernel': None, # default: RBF
    }

In [ ]:
logger = JSONLogger(path="./logs3.json")
dqn_bo.subscribe(Events.OPTMIZATION_STEP, logger)

In [ ]:
# initalize
dqn_bo.maximize(
    init_points=10,
    n_iter=0,
)

In [ ]:
# acquisition function
utility = UtilityFunction(kind="ucb", kappa=2.5, xi=0.0)

In [ ]:
print(dqn_bo.max)

In [ ]:
for _ in range(40):
    next_point = dqn_bo.suggest(utility)
    target = black_box_function(**next_point)
    dqn_bo.register(params=next_point, target=target)
    print(target, next_point)
print(dqn_bo.max)

In [ ]:
import pandas as pd
import numpy as np
np.set_printoptions(precision=4)

In [ ]:
ys = np.reshape(dqn_bo._space.target,newshape=(50,1))
xs = dqn_bo._space.params
arrs = np.append(xs,ys,axis=1)

In [ ]:
df = pd.DataFrame(arrs)
df = df.rename(columns={0:'e_greedy',1:'learning_rate',2:'function_value'})

In [ ]:
df.to_csv('result.csv')

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy import interpolate

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
X = arrs[:,0]
Y = arrs[:,1]
Z = arrs[:,2]
ax.scatter(X,Y,Z)
plt.show()